In [34]:
import sqlite3
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

In [35]:
# === 1. ЗАГРУЗКА ДАННЫХ ===
db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day.db')

with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        "SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME FROM Futures",
        conn
    )

df_fut

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME
0,2015-01-05,76930.0,72470.0,78980.0,74600.0,372848
1,2015-01-06,74470.0,71200.0,74610.0,73480.0,319307
2,2015-01-08,73490.0,71000.0,81380.0,79980.0,537469
3,2015-01-09,79950.0,74450.0,81050.0,77650.0,592715
4,2015-01-12,77210.0,73180.0,77550.0,73900.0,440908
...,...,...,...,...,...,...
2537,2025-02-12,98600.0,98120.0,101480.0,100700.0,124069
2538,2025-02-13,100570.0,100080.0,111010.0,109930.0,247083
2539,2025-02-14,109750.0,106470.0,114130.0,108620.0,282242
2540,2025-02-17,108960.0,108050.0,115370.0,115170.0,183356


In [36]:
# === 2. ФУНКЦИЯ КОДИРОВАНИЯ СВЕЧЕЙ (ЛИХОВИДОВ) ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']

    if close > open_:
        direction = 1  
    elif close < open_:
        direction = 0  
    else:
        direction = 2  

    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)

    def classify_shadow(shadow, body):
        if shadow < 0.1 * body:
            return 0  
        elif shadow < 0.5 * body:
            return 1  
        else:
            return 2  

    upper_code = classify_shadow(upper_shadow, body)
    lower_code = classify_shadow(lower_shadow, body)

    return f"{direction}{upper_code}{lower_code}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === 3. ПОДГОТОВКА ДАННЫХ ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

df_fut['direction_candle_code'] = np.nan

df_fut

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,CANDLE_CODE,CANDLE_INT,direction_candle_code
0,2015-01-05,76930.0,72470.0,78980.0,74600.0,372848,022,8,NaN
1,2015-01-06,74470.0,71200.0,74610.0,73480.0,319307,012,5,NaN
2,2015-01-08,73490.0,71000.0,81380.0,79980.0,537469,111,13,NaN
3,2015-01-09,79950.0,74450.0,81050.0,77650.0,592715,012,5,NaN
4,2015-01-12,77210.0,73180.0,77550.0,73900.0,440908,011,4,NaN
...,...,...,...,...,...,...,...,...,...
2537,2025-02-12,98600.0,98120.0,101480.0,100700.0,124069,111,13,NaN
2538,2025-02-13,100570.0,100080.0,111010.0,109930.0,247083,110,12,NaN
2539,2025-02-14,109750.0,106470.0,114130.0,108620.0,282242,022,8,NaN
2540,2025-02-17,108960.0,108050.0,115370.0,115170.0,183356,101,10,NaN


In [37]:
# Сохранение в список
trade_dates = df_fut['TRADEDATE'].tolist()

print(trade_dates)

['2015-01-05', '2015-01-06', '2015-01-08', '2015-01-09', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19', '2015-03-20', '2015-03-23', '2015-03-24', '2015-03-25', '2015-03-26', '2015-03-27', '2015-03-30', '2015-03-31', '2015-04-01', '2015-04-02', '2015-04-03', '2015-04-06', '2015-04-07', '2015-04-08', '2015-04-09', '2015-04-10', '2015-04-13', '2015-04-14', '2015-04-15', '2015-04-16', '2015

Разбиение дата фрейма 80/20

In [38]:
split_index = int(len(df_fut) * 0.8)  # Вычисляем индекс разбиения

df_80 = df_fut.iloc[:split_index]  # Первые 80%
df_20 = df_fut.iloc[split_index:]  # Последние 20%

# Получаем значение из 0-й строки df_20
first_trade_date = df_20.iloc[0]['TRADEDATE']

print(first_trade_date)


2023-02-17


In [39]:
# === 4. СОЗДАНИЕ НЕЙРОСЕТИ (LSTM) ===
class CandleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(CandleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  
        return self.sigmoid(x)

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CandleLSTM(
    vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=1
    ).to(device)

# Загружаем модель
model.load_state_dict(torch.load("best_model_02.pth"))
model.eval()

# Разделяем 80/20 и получаем first_trade_date
split_index = int(len(df_fut) * 0.8)
df_20 = df_fut.iloc[split_index:]
first_trade_date = df_20.iloc[0]['TRADEDATE']

# Определяем начальный индекс так, чтобы последнее значение первого окна было first_trade_date
start_index = df_fut[df_fut['TRADEDATE'] == first_trade_date].index[0] - 24  # 25 строк = 24 шага назад

# Если индекс ушел в минус, корректируем
start_index = max(0, start_index)

# Итерация по окнам в 25 строк
for i in range(start_index, len(df_fut) - 24):
    window = df_fut.iloc[i:i+25]  # Берем окно в 25 строк
    # print(f"Окно {i} - {i+24}, Последняя дата: {window.iloc[-1]['TRADEDATE']}")

    # Берем последние 20 свечей из df_fut
    last_sequence = torch.tensor(
        window['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
        ).unsqueeze(0).to(device)

    # Предсказание
    with torch.no_grad():
        probability_up = model(last_sequence).item()  # Вероятность роста
        probability_down = 1 - probability_up  # Вероятность падения

        direction = "📈 UP" if probability_up >= 0.5 else "📉 DOWN"

        print(f"Prediction: {direction}")
        print(f"UP Probability: {probability_up:.2%}, DOWN Probability: {probability_down:.2%}")
        print()

Prediction: 📈 UP
UP Probability: 95.67%, DOWN Probability: 4.33%

Prediction: 📉 DOWN
UP Probability: 1.00%, DOWN Probability: 99.00%

Prediction: 📉 DOWN
UP Probability: 0.00%, DOWN Probability: 100.00%

Prediction: 📉 DOWN
UP Probability: 25.27%, DOWN Probability: 74.73%

Prediction: 📈 UP
UP Probability: 99.97%, DOWN Probability: 0.03%

Prediction: 📉 DOWN
UP Probability: 35.54%, DOWN Probability: 64.46%

Prediction: 📈 UP
UP Probability: 96.25%, DOWN Probability: 3.75%

Prediction: 📉 DOWN
UP Probability: 1.71%, DOWN Probability: 98.29%

Prediction: 📈 UP
UP Probability: 99.54%, DOWN Probability: 0.46%

Prediction: 📈 UP
UP Probability: 89.56%, DOWN Probability: 10.44%

Prediction: 📈 UP
UP Probability: 100.00%, DOWN Probability: 0.00%

Prediction: 📈 UP
UP Probability: 54.88%, DOWN Probability: 45.12%

Prediction: 📈 UP
UP Probability: 83.86%, DOWN Probability: 16.14%

Prediction: 📈 UP
UP Probability: 100.00%, DOWN Probability: 0.00%

Prediction: 📈 UP
UP Probability: 99.89%, DOWN Probability: